#### Import statements

In [1]:
import torch
import shutil
import os
import numpy as np

#### Load the drive folder containing all required files

In [2]:
# mount the drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# access the drive folder containing everything we need
%cd /content/drive/My Drive/Colab environments/Risiko! DL

# check that we are in the desired folder
%ls

Mounted at /content/drive
/content/drive/My Drive/Colab environments/Risiko! DL
 3D_models/                                 runs/
 backgrounds/                               Split_train_test_val.ipynb
 coco_risiko.yaml                           synthetic_dataset/
 custom_yolo.yaml                           synthetic_images/
 datasets/                                  tanks_flags_detection.ipynb
 pre_trained_weights/                       Test_detection.ipynb
 real_images/                               test_example.txt
'Risiko!_Synthetic_Dataset_Creator.ipynb'   test.txt
'Risiko! Test.ipynb'                        yolov5/


In [3]:
# generic path to the weights folder
weights_folder = 'runs/train'
weights_path = os.path.join(os.getcwd(), weights_folder)
print(weights_path)

# specific path to weigths obtained with 300 epochs
specific_folder = 'exp_300_epochs/weights/best.pt'
best_weights_path = os.path.join(weights_path, specific_folder)
print(best_weights_path)

/content/drive/My Drive/Colab environments/Risiko! DL/runs/train
/content/drive/My Drive/Colab environments/Risiko! DL/runs/train/exp_300_epochs/weights/best.pt


#### Clone the GitHub repository yolov5 and install requirements

In [4]:
print(os.getcwd())
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

/content/drive/My Drive/Colab environments/Risiko! DL
fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/drive/My Drive/Colab environments/Risiko! DL/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.0/613.0 kB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.6 MB/s eta 0:00:00


#### Model definition

In [5]:
# set the device
device = torch.device("cuda" if torch.cuda.is_available else "cpu")

# model
print(os.getcwd())
model = torch.hub.load(os.getcwd(), 'custom', path = best_weights_path, source ='local', force_reload=True)
model.to(device)

/content/drive/My Drive/Colab environments/Risiko! DL/yolov5


YOLOv5 🚀 2023-6-11 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7042489 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

####Intersection over Union (IoU)
We use this method in the section below during the evaluation of the model

In [6]:
# Compute the IOU between bboxes $bbox_1 and $bbox_2.
# $bbox_1 and $bbox_2 are in [x_min, y_min, x_max, y_max] format.
# Return the IOU between $bbox_1 and $bbox_2.
def calculate_iou(bbox_1, bbox_2):

    # Compute the intersection coordinates
    x_left = max(bbox_1[0], bbox_2[0])
    y_top = max(bbox_1[1], bbox_2[1])
    x_right = min(bbox_1[2], bbox_2[2])
    y_bottom = min(bbox_1[3], bbox_2[3])

    # compute the areas of both bounding bboxes
    area_bbox_1 = (bbox_1[2] - bbox_1[0] + 1) * (bbox_1[3] - bbox_1[1] + 1)
    area_bbox_2 = (bbox_2[2] - bbox_2[0] + 1) * (bbox_2[3] - bbox_2[1] + 1)

    # compute the area of the intersection
    intersection_area = max(0, (x_right - x_left + 1)) * max(0, (y_bottom - y_top + 1))

    # calculate the IOU
    return intersection_area / float(area_bbox_1 + area_bbox_2 - intersection_area)

####Inference on the synthetic test set

In [7]:
# Set the model in evaluation mode
model.eval()

# Path to our dataset
path = '/content/drive/My Drive/Colab environments/Risiko! DL/datasets/test/synthetic/images'

# Get the list of files in the directory
files = os.listdir(path)

# create a list of test images: batch for inference
test_synthetic_images = []

for file_name in files:
    # full file path
    file_path = os.path.join(path, file_name)
    # add the image to the list
    test_synthetic_images.append(file_path)

####Load the ground truth labels

In [8]:
# Convert the input bbox from [x_center, y_center, width, height] format to
# [x_min, y_min, x_max, y_max] format.
def convert_bbox(bbox):

    x_center = bbox[0]
    y_center = bbox[1]
    width = bbox[2]
    height = bbox[3]

    x_min = x_center - width / 2
    y_min = y_center - height / 2
    x_max = x_center + width / 2
    y_max = y_center + height / 2

    return [x_min, y_min, x_max, y_max]

In [9]:
# path to the labels
labels_path = '/content/drive/My Drive/Colab environments/Risiko! DL/datasets/test/synthetic/labels'

# Get the list of files in the directory
files = os.listdir(labels_path)

# create a list of test labels
test_synthetic_labels = []

for file_name in files:
    # full file path
    file_path = os.path.join(labels_path, file_name)
    # add the image to the list
    test_synthetic_labels.append(file_path)

# store the true bboxes values
true_bboxes = []

# store the true class label values
true_labels = []

# iterate over the labels
for label_file in test_synthetic_labels:
    # open the current file
    with open(label_file, "r") as f:
        # class label values in the current file
        current_file_classes = []
        # bboxes values in the current file
        current_file_bboxes = []

        # iterate over the lines: each line is associated with a true instance
        for line in f:
            # extract values from a line
            string_values = line.split()
            # append the class value in the current line of the current file
            current_file_classes.append(float(string_values[0]))
            # values of the bbox in the current line of the current file
            bbox = []
            for i in range(1, len(string_values)):
                bbox.append(float(string_values[i]))
            # convert the bbox format from (x_center, y_center, width, height)
            # to (x_min, y_min, x_max, y_max)
            conv_bbox = convert_bbox(bbox)
            # append the bbox in the current line of the current file
            current_file_bboxes.append(conv_bbox)

        # append the bboxes related to the current file
        true_bboxes.append(current_file_bboxes)
        # append the class labels related to the current file
        true_labels.append(current_file_classes)

####Evaluation: mean IOU

In [10]:
IMAGES_WIDTH = 1920
IMAGES_HEIGHT = 1280

# Normalize a bbox in image of size IMAGES_WIDTH x IMAGES_HEIGHT to values in [0, 1].
# The bbox must be in format [x_min, y_min, x_max, y_max].
# Return the normalized bbox.
def normalize_predicted_bbox(bbox):

    x_min = bbox[0] / IMAGES_WIDTH
    y_min = bbox[1] / IMAGES_HEIGHT
    x_max = bbox[2] / IMAGES_WIDTH
    y_max = bbox[3] / IMAGES_HEIGHT

    return [x_min, y_min, x_max, y_max]

In [11]:
# Compute the mean IOU for each class between predicted bboxes and true ones in a given image.
# $predicted_bboxes: predicted bboxes in the image.
# $true_bboxes: ground truth bboxes in the image.
# $predicted_classes: predicted class labels for each bbox in the image.
# $true_classes: true class labels of true bboxes in the image.
# Return a dictionary $class_mean_IOU with classes as keys and mean IOU score for that class in the image.
def class_mean_IOU_image(predicted_bboxes, true_bboxes, predicted_classes, true_classes):

    # dictionary where each key is a class label and the value is the list of IOU for that label
    class_IOU_list = {}

    # iterate over the predicted bboxes
    for i in range(len(predicted_bboxes)):

        # list of IOU scores between the current predicted bbox and all the true bboxes with the same class
        current_bbox_IOU_scores = []

        # class label of the current predicted bbox
        predicted_class = predicted_classes[i].tolist()

        # initialize the entry of the dictionary
        class_IOU_list[predicted_class] = []

        # iterate over the true bboxes to compute the IOU scores for the current predicted bbox
        for j in range(len(true_bboxes)):
            if predicted_class == true_classes[j]:
                current_bbox_IOU_scores.append(
                    calculate_iou(normalize_predicted_bbox(predicted_bboxes[i].tolist()), true_bboxes[j]))
        if len(current_bbox_IOU_scores) > 0:
            class_IOU_list[predicted_class].append(np.max(current_bbox_IOU_scores))
        else:
            class_IOU_list[predicted_class].append(0)

    # dictionary where each class label is a key and the associated value is the mean IOU with that label
    class_mean_IOU = {}

    # fill the dictionary
    for class_label in class_IOU_list.keys():
        class_mean_IOU[class_label] = np.mean(class_IOU_list[class_label])

    return class_mean_IOU


# Compute the mean IOU for each class between predicted bboxes and true ones for a dataset of images.
# $test_images is a set of paths to images.
# $yolo_model is a YOLO model to extract predictions from $test_images.
# $true_bboxes: ground truth bboxes. $true_bboxes[i] contains true bboxes in image $i in $test_images.
# $true_classes: true class labels of true bboxes. $true_classes[i] contains the class label of $true_bboxes[i].
# Return a dictionary $class_mean_IOU with classes as keys and mean IOU score for that class in $test_images.
def class_mean_IOU(test_images, yolo_model, true_bboxes, true_classes):

    # dictionary with class labels as keys and the corresponding lists of IOU scores as values
    IOU_scores = {}

    # iterate over the images
    for i in range(len(test_images)):

        # inference on the current batch of images
        results = yolo_model(test_images, size=640)

        # result of inference on image $i in the batch
        predictions = results.xyxy[i]
        # predicted bboxes
        pred_boxes = predictions[:, :4] # [xmin, ymin, xmax, ymax] for each bbox
        # corresponding predicted class labels
        pred_labels = predictions[:, 5] # label for each prediction in the image

        # dictionary of mean IOU values for the current image
        current_image_IOU = class_mean_IOU_image(pred_boxes, true_bboxes[i], pred_labels, true_classes[i])

        # iterate over the dictionary to fill the global list
        for class_label in current_image_IOU.keys():
            if class_label not in IOU_scores:
                IOU_scores[class_label] = []
            IOU_scores[class_label].append(current_image_IOU[class_label])

    # compute the mean IOU for each class and return it
    class_mean_IOU = {}
    for class_label in IOU_scores.keys():
        class_mean_IOU[class_label] = np.mean(IOU_scores[class_label])

    return class_mean_IOU


# Compute the mean IOU, considering all class mean IOU values.
# $classes_mean_IOU is the dictionary containing the class labels as keys and related class mean IOU as value.
# Return the value $mean_IOU: mean IOU considering each class mean IOU.
def mean_IOU(classes_mean_IOU):

    # sum IOU over all classes
    sum_IOU = 0

    # number of labels with mean IOU different from 0
    num_classes = 0

    # iterate over the dictionary
    for class_label in classes_mean_IOU.keys():
        sum_IOU += classes_mean_IOU[class_label]
        num_classes += 1

    return sum_IOU / num_classes

In [18]:
# Print all (key, value) pairs in the dictionary $dic.
# The dictionary must have float numbers as values.
# As a first row, the string $s is printed before the dictionary.
def print_dictionary(dic, s):
    print(s)
    for key in dic.keys():
        print(str(key) + ": " + "{:.3f}".format(dic[key]))

In [20]:
# number of images to be considered for the evaluation
N_IMAGES = 30

# we compute the score only for the first 20 images due to the computational effort required
class_mean_iou = class_mean_IOU(test_synthetic_images[:N_IMAGES], model, true_bboxes, true_labels)
# sort the dictionary by key, i.e. by class
class_mean_iou = dict(sorted(class_mean_iou.items()))
print_dictionary(class_mean_iou, "Mean IOU for each class:")

mean_iou = mean_IOU(class_mean_iou)
print("Mean IOU over all classes: " + "{:.3f}".format(mean_iou))

Mean IOU for each class:
0.0: 0.962
1.0: 0.983
2.0: 0.947
3.0: 0.963
4.0: 0.963
5.0: 0.949
6.0: 0.955
7.0: 0.937
8.0: 0.946
9.0: 0.982
10.0: 0.956
11.0: 0.988
Mean IOU over all classes: 0.961
